In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk import word_tokenize
import nltk
import sys
from sklearn.feature_extraction.text import CountVectorizer
import scipy

%matplotlib inline

In [2]:
# !wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv -P ./data

In [3]:
# nltk.download('punkt')

In [4]:
data = pd.read_csv('./data/bbc_text_cls.csv')
print(data.shape)
data.head()

(2225, 2)


,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [5]:
word2idx = {}
tokened_docs = []
idx_count = 0
for text in data['text'].tolist():
    tokens = word_tokenize(text.lower())
    tokened_doc = []
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx_count
            idx_count += 1
        tokened_doc.append(word2idx[token])
    tokened_docs.append(tokened_doc)

In [6]:
# countVec = CountVectorizer()
# X = countVec.fit_transform(data['text'].tolist())
# X

In [7]:
idx2word = {v:k for k,v in word2idx.items()}

In [8]:
N = len(tokened_docs) ## Number of documents
V = len(word2idx) ## Vocabulary size

print(N,V)
tf_array = scipy.sparse.dok_matrix((N,V),dtype = np.float32)

2225 34762


In [9]:
for i,_doc in enumerate(tokened_docs):
    for token in _doc:
        tf_array[i,token] += 1

In [10]:
print(sys.getsizeof(tf_array)/(1024*1024))

20.000099182128906


In [11]:
doc_freq = np.sum(tf_array > 0,axis=0)
idf = np.log(N/doc_freq)

In [12]:
# tf_array_csr = tf_array.tocsr()
print(tf_array.shape,idf.shape)

(2225, 34762) (1, 34762)


In [13]:
tfidf_array = tf_array.multiply(idf)
type(tfidf_array)

scipy.sparse._coo.coo_matrix

In [14]:
tfidf_array = tfidf_array.todok()

In [15]:
### Choose random docs and get terms with highest tfidf scores in doc

In [19]:
rand_idx = np.random.choice(N)
text,label = data.loc[rand_idx,'text'],data.loc[rand_idx,'labels']
print(text.split('\n')[0],' : ',label)
array_row = tfidf_array[rand_idx].toarray().flatten()
sorted_idxs = np.argsort(-array_row)
top5 = sorted_idxs[:5]
for idx in top5:
    print(idx2word[idx],' : ',array_row[idx])

Howard denies split over ID cards  :  politics
id  :  44.116753285960115
cards  :  35.966383304270295
howard  :  16.63822956881789
shadow  :  13.252252159711608
yeo  :  12.6424356669609
